In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
from copy import deepcopy
steps={
    '<' : (0,-1),
    '>' : (0,1),
    'v' : (1,0),
    '^' : (-1,0),
}
def take_step(fmap, pos, step):
    y,x = pos
    cy, cx = steps[step]
    nmove=1
    while fmap[y+nmove*cy][x+nmove*cx] in ['O', '[', ']'] :
        nmove+=1
    if fmap[y+nmove*cy][x+nmove*cx] == '#':
        return fmap, pos
    if fmap[y+nmove*cy][x+nmove*cx] == '.':
        new_map = deepcopy(fmap)
        for t in range(nmove):
            new_map[y+(t+1)*cy][x+(t+1)*cx] = fmap[y+t*cy][x+t*cx]
        new_map[y][x] = '.'
        
        return new_map, (y+cy, x+cx)

In [3]:
double={'#':'##',
        '.':'..',
        'O':'[]',
        '@': '@.',
       }


def read_file(path,doubling=False):
    fishmap=[]
    fish_path=''
    start =None
    reading_map = True
    with open(path,'r') as f:
        for y,line in enumerate(f.readlines()):
            line=line.strip()
            if len(line) < 2:
                reading_map=False
                continue
            if reading_map:
                if doubling:
                    line = ''.join([ double[l] for l in line])
                if '@' in line:
                    x= line.index('@')
                    start=(y,x)
                row=list(line)
                fishmap.append(row)
            else:
                fish_path = fish_path+line
    return fishmap, fish_path,start

In [4]:
def score_map(fishmap):
    score = 0
    for y,row in enumerate(fishmap):
        for x,v in enumerate(row):
            if v == 'O':
                score += 100*y+x
            elif v =='[':
                score += 100*y+x
    return score

In [5]:
def part_one(path='input_data/test_15.txt'):
    fishmap,fish_path,start = read_file(path)

    pos = start
    for step in fish_path:
        fishmap, pos = take_step(fishmap, pos, step)

    
    print( '\n'.join([ ''.join(r) for r in fishmap]))
    return score_map(fishmap)

In [6]:
part_one()

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########


10092

In [7]:
part_one('input_data/day_15.txt')

##################################################
#.O....O..OO...O#..OOO.OOOO.O#.##.#OOOOO.....OOOO#
##....##.......OO#.O.O#OOO..OOO.....OO.....O#...O#
#.....OO........OO.OOOOOOO........O.O......#.#..O#
#.O#O.OOO...#.#......O.OOOO.....#.O.OO#....O....O#
#.OOOOO.....#........OOO.OOOOO..O.O.OOO.OO.....OO#
#..#OOOO................OO.#..O.OOO#O#.O#O......O#
##..O..O#O.....O.....#..O.OOOOOOOOO..O.#O........#
#......OOO......O..........OO.O.O#....O..........#
#.........#........OO.O......#........O#O....#...#
#.....O.OOOO...#.#.#O#.........#......O...O..O.#.#
#O..O.OO#OO..#OO...OO.O.....O.........#..#...O...#
#O....OO..........#O................O........OO.O#
#.#O...OO....................O#OO............OOOO#
##OO.....#O##..O......#....OOOOOO.#.......O##.#O##
#OOO..#....#OOOO.#......O.###OO#..O....O.....O.OO#
#OOOO..O...O.OO#..O..OOO...OO#OOOO.O.O.#.......OO#
#..O..#....OOOO....OO#.........OO#.O.O.....#..O#.#
#....O...OOOOOO.....#O.....#...#OOOOO....O....OO##
#O........#...O......OOOO#..#..

1349898

# Part 2

In [8]:
def apply_vertical_shift(fmap, step, pos_move):
    new_map = deepcopy(fmap)
    cy, cx = steps[step]

    for pos in pos_move:
        y,x = pos
        new_map[y][x] = '.'
    
    for pos in pos_move:
        y,x = pos
        new_map[y+cy][x+cx] = fmap[y][x]

    return new_map

def vertical_box_step(fmap, pos, step, works, pos_moved):
    y,x = pos
    cy, cx = steps[step]
    
    n_sym = fmap[y+cy][x+cx]
    if n_sym == "#":
        works.append(False)
        pos_moved.clear()

    pos_moved.append(pos)
    works.append(True)
    
    if n_sym == "]":
        vertical_box_step(fmap, (y+cy,x+cx), step, works, pos_moved)
        vertical_box_step(fmap, (y+cy,x+cx-1), step, works, pos_moved)
    
    if n_sym == "[":
        vertical_box_step(fmap, (y+cy,x+cx), step, works, pos_moved)
        vertical_box_step(fmap, (y+cy,x+cx+1), step, works, pos_moved)
    


def take_box_step(fmap, pos, step):
    if step in ['>','<']:
        return take_step(fmap, pos, step)

    y,x = pos
    cy, cx = steps[step]
    works =[]
    pos_moved=[]
    
    vertical_box_step(fmap, pos, step, works, pos_moved)
    if all(works): 
        new_map = apply_vertical_shift(fmap, step, pos_moved)
        return new_map, (y+cy, x+cx)
    
    return fmap, pos
    

In [9]:
def part_two(path='input_data/test_15.txt'):
    fishmap,fish_path,start = read_file(path,doubling=True)

    pos = start
    for step in fish_path:
        fishmap, pos = take_box_step(fishmap, pos, step)
    
    print( '\n'.join([ ''.join(r) for r in fishmap]))
    return score_map(fishmap)

In [10]:
part_two()

####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################


9021

In [11]:
part_two('input_data/day_15.txt')


####################################################################################################
##..[]...[]...[][][]............##....[][][]..[][][][]..[]##[]####..##..[]......[]......[]..[][]..##
####....[]..####[][]............[]##.[][].[]##[]..[][][][][][][][][]......[]..[][]....[]##......[]##
##......[][][]....................[].[].[][].....[]...........[][][]..[][]......[]....##..##....[]##
##..[]##[].[]...........##..##..................................##[][]....[]##[].....[]...........##
##[][][]...[]...........##.....[].........[]....[]..............[]........[]......................##
##....##........................[][]......[]..........##............[]##[]##..[]##[]..............##
####[][]........##..........[]............##...[][]...[].............[][][][].##[]................##
##...[]......[].[]...[][]..[]..........[].........................##....[][][][][][]......[][]....##
##....[]..........[]##..[].[].[]......[]..........[]......##...........[].[][]##[][][][]..#

1376686